In [ ]:
!pip -qq install torchutils

In [ ]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from torchvision import transforms as T

# Для чтения изображений с диска
from torchvision import io # input/output
import torchutils as tu
import json
import numpy as np
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import vgg16, VGG16_Weights

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

cpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
zip_file_path = '/content/drive/MyDrive/Colab Notebooks/archive.zip'

In [ ]:
import zipfile
import os

# Укажите путь к вашему архиву
extract_path = '/content/dataset/'  # Путь для разархивированных файлов

# Создайте директорию, если она не существует
os.makedirs(extract_path, exist_ok=True)

# Разархивируйте файл
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Разархивирование завершено!")

Разархивирование завершено!


In [ ]:
trnsfrms = T.Compose(
    [
        T.Resize((224, 224)),
        T.ToTensor()
    ]
)

In [ ]:
dataset = torchvision.datasets.ImageFolder(
    '/content/dataset/train',
    transform=trnsfrms)

In [ ]:
train_dataset, valid_dataset = random_split(dataset, lengths=(.7, .3))

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True)

In [ ]:
img = io.read_image('/content/dataset/test/altar/220.jpg')/255
img.shape

torch.Size([3, 64, 64])

In [ ]:
class MyResNet(nn.Module):
    def __init__(self):
        super().__init__()

        # подгружаем модель
        self.model = vgg16(weights=VGG16_Weights.DEFAULT)
        # заменяем слой
        self.model.fc = nn.Linear(2048, 10)
        # замораживаем слои
        for i in self.model.parameters():
            i.requires_grad = False
        # размораживаем только последний, который будем обучать
        self.model.fc.weight.requires_grad = True
        self.model.fc.bias.requires_grad = True

    def forward(self, x):
        return self.model(x)

model = MyResNet()
model.to(DEVICE);

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
tu.get_model_summary(model, torch.randn(4, 3, 64, 64, device=DEVICE))

Layer                                               Kernel              Output         Params          FLOPs
0_model.Conv2d_conv1                               [3, 64, 7, 7]    [4, 64, 32, 32]       9,408   38,535,168
1_model.BatchNorm2d_bn1                                     [64]    [4, 64, 32, 32]         128    1,048,576
2_model.ReLU_relu                                              -    [4, 64, 32, 32]           0            0
3_model.MaxPool2d_maxpool                                      -    [4, 64, 16, 16]           0            0
4_model.layer1.0.Conv2d_conv1                     [64, 64, 1, 1]    [4, 64, 16, 16]       4,096    4,194,304
5_model.layer1.0.BatchNorm2d_bn1                            [64]    [4, 64, 16, 16]         128      262,144
6_model.layer1.0.ReLU_relu                                     -    [4, 64, 16, 16]           0            0
7_model.layer1.0.Conv2d_conv2                     [64, 64, 3, 3]    [4, 64, 16, 16]      36,864   37,748,736
8_model.layer1.0.Ba

In [ ]:
train_epoch_acc = []
train_epoch_losses = []
valid_epoch_losses = []
valid_epoch_acc = []

for epoch in range(3):
    model.train()
    loss_batch = []
    acc_batch  = []

    for images, labels in train_loader:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        preds = model(images)  # Убедитесь, что preds имеет размерность [batch_size, num_classes]

        # Вычисление потерь
        loss = criterion(preds, labels.long())  # Убедитесь, что labels имеют правильный тип
        loss_batch.append(loss.item())

        # Вычисление точности
        predicted = torch.argmax(preds, dim=1)  # Получаем индексы классов с максимальной вероятностью
        accuracy = (predicted == labels).cpu().numpy().mean()  # Сравниваем предсказанные классы с метками
        acc_batch.append(accuracy)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_epoch_losses.append(np.mean(loss_batch))
    train_epoch_acc.append(np.mean(acc_batch))

    model.eval()
    loss_batch = []
    acc_batch  = []
    for images, labels in valid_loader:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        with torch.no_grad():
            preds = model(images)  # Убедитесь, что preds имеет размерность [batch_size, num_classes]

        loss = criterion(preds, labels.long())  # Убедитесь, что labels имеют правильный тип
        loss_batch.append(loss.item())

        # Вычисление точности
        predicted = torch.argmax(preds, dim=1)  # Получаем индексы классов
        accuracy = (predicted == labels).cpu().numpy().mean()  # Сравниваем предсказанные классы с метками
        acc_batch.append(accuracy)

    valid_epoch_losses.append(np.mean(loss_batch))
    valid_epoch_acc.append(np.mean(acc_batch))

    print(f'Epoch: {epoch}  loss_train: {train_epoch_losses[-1]:.3f}, loss_valid: {valid_epoch_losses[-1]:.3f}')
    print(f'\t  metrics_train: {train_epoch_acc[-1]:.3f}, metrics_valid: {valid_epoch_acc[-1]:.3f}')


KeyboardInterrupt: 

In [ ]:
test_dataset = torchvision.datasets.ImageFolder(
    '/content/dataset/test',
    transform=trnsfrms
)

NameError: name 'torchvision' is not defined